# Exercise Set 12: Linear regression models.

*Afternoon, August 19, 2019*

In this Exercise Set 12 we will work with linear regression models.

We import our standard stuff. Notice that we are not interested in seeing the convergence warning in scikit-learn so we suppress them for now.

In [9]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import seaborn as sns

%matplotlib inline

## Exercise Section 12.1: Estimating linear models with gradient decent
 
Normally we use OLS to estimate linear models. In this exercise we replace the OLS-estimator with a new estimator that we code up from scratch. We solve the numerical optimization using the gradient decent algorithm. Using our algorithm we will fit it to some data, and compare our own solution to the standard solution from `sklearn`

> **Ex. 12.1.0**: Import the dataset `tips` from the `seaborn`.


*Hint*: use the `load_dataset` method in seaborn

In [2]:
# [Answer to Ex. 12.1.0]
import seaborn as sns

tips = sns.load_dataset('tips')

tips.head()


,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


> **Ex. 12.1.1**: Convert non-numeric variables to dummy variables for each category (remember to leave one column out for each catagorical variable, so you have a reference). Restructure the data so we get a dataset `y` containing the variable tip, and a dataset `X` containing the 
features. 

>> *Hint*: You might want to use the `get_dummies` method in pandas, with the `drop_first = True` parameter. 

In [24]:
# [Answer to Ex. 12.1.1]
import pandas as pd

X = pd.get_dummies(tips, columns=["sex","smoker", "day", "time"], drop_first=True)
Y = X["tip"]
X = X.drop("tip", axis=1)
Y
X
#type(X)
#type(Y)

,total_bill,size,sex_Female,smoker_No,day_Fri,day_Sat,day_Sun,time_Dinner
0,16.99,2,1,1,0,0,1,1
1,10.34,3,0,1,0,0,1,1
2,21.01,3,0,1,0,0,1,1
3,23.68,2,0,1,0,0,1,1
4,24.59,4,1,1,0,0,1,1
5,25.29,4,0,1,0,0,1,1
6,8.77,2,0,1,0,0,1,1
7,26.88,4,0,1,0,0,1,1
8,15.04,2,0,1,0,0,1,1
9,14.78,2,0,1,0,0,1,1


> **Ex. 12.1.2**: Divide the features and target into test and train data. Make the split 50 pct. of each. The split data should be called `X_train`, `X_test`, `y_train`, `y_test`.

>> *Hint*: You may use `train_test_split` in `sklearn.model_selection`.

In [25]:
# [Answer to Ex. 12.1.2]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.5, random_state=0)

> **Ex. 12.1.3**: Normalize your features by converting to zero mean and one std. deviation.

>> *Hint 1*: Take a look at `StandardScaler` in `sklearn.preprocessing`. 

>> *Hint 2*: If in doubt about which distribution to scale, you may read [this post](https://stats.stackexchange.com/questions/174823/how-to-apply-standardization-normalization-to-train-and-testset-if-prediction-i).

In [39]:
# [Answer to Ex. 12.1.3]
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_x = scaler.fit_transform(X_train)
scaled_x


array([[-0.40464265,  1.37336328, -0.71145825,  0.73776947, -0.26490647,
        -0.73776947,  1.60879933,  0.67259271],
       [ 0.0992668 ,  0.37603995, -0.71145825,  0.73776947, -0.26490647,
        -0.73776947,  1.60879933,  0.67259271],
       [ 3.13833134,  1.37336328, -0.71145825,  0.73776947, -0.26490647,
         1.35543694, -0.62158156,  0.67259271],
       [-1.4057725 , -0.62128339, -0.71145825,  0.73776947, -0.26490647,
        -0.73776947, -0.62158156, -1.48678388],
       [ 0.03683554,  1.37336328, -0.71145825,  0.73776947, -0.26490647,
         1.35543694, -0.62158156,  0.67259271],
       [-1.16385137, -0.62128339, -0.71145825,  0.73776947, -0.26490647,
        -0.73776947,  1.60879933,  0.67259271],
       [-0.03451447, -0.62128339,  1.40556386, -1.35543694, -0.26490647,
        -0.73776947, -0.62158156, -1.48678388],
       [-0.77588568, -0.62128339,  1.40556386,  0.73776947, -0.26490647,
        -0.73776947, -0.62158156, -1.48678388],
       [-0.03897385,  1.37336328

> **Ex. 12.1.4**: Make a function called `compute_error` to compute the prediction errors given input target `y_`, input features `X_` and input weights `w_`. You should use matrix multiplication.
>
>> *Hint 1:* You can use the net-input fct. from yesterday.
>>
>> *Hint 2:* If you run the following code,
>> ```python
y__ = np.array([1,1])
X__ = np.array([[1,0],[0,1]])
w__ = np.array([0,1,1])
compute_error(y__, X__, w__)
```

>> then you should get output:
```python 
array([0,0])
```



In [44]:
# [Answer to Ex. 12.1.4]

def net_input(X,w):
    z = w[0] + X.dot(w[1:])
    return z

def compute_error(y,x,w):
    e = y-net_input(x,w)
    return e

y__ = np.array([1,1])
X__ = np.array([[1,0],[0,1]])
w__ = np.array([0,1,1])
compute_error(y__,X__, w__)

array([0, 0])

> **Ex. 12.1.5**: Make a function to update the weights given input target `y_`, input features `X_` and input weights `w_` as well as learning rate, $\eta$, i.e. greek `eta`. You should use matrix multiplication.

In [51]:
# [Answer to Ex. 12.1.5]
eta = 0.1

def function(y_, x_, w_, eta):
    errors = 0
    for (xi, yi) in zip(X_train,y_train):
        update = eta * X.T.dot(compute_error(y, X, W))
        W[1:] = W[1:] + update * xi
        W[0] = W[0] + eta*compute_error(y, X, w)
        errors = errors + int(update != 0) 
        
    return W, errors

a = function(y__, X__, w__, eta)

NameError: name 'predict' is not defined

> **Ex. 12.1.6**: Use the code below to initialize weights `w` at zero given feature set `X`. Notice how we include an extra weight that includes the bias term. Set the learning rate `eta` to 0.001. Make a loop with 50 iterations where you iteratively apply your weight updating function. 

>```python
w = np.zeros(1+X.shape[1])
```

In [13]:
# [Answer to Ex. 12.1.6]

> **Ex. 12.1.7**: Make a function to compute the mean squared error. Alter the loop so it makes 100 iterations and computes the MSE for test and train after each iteration, plot these in one figure. 

>> Hint: You can use the following code to check that your model works:
>>```python
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_train, y_train)
assert((w[1:] - reg.coef_).sum() < 0.01)
```

In [16]:
# [Answer to Ex. 12.1.7]

The following bonus exercises are for those who have completed all other exercises until now and have a deep motivation for learning more.

> **Ex. 12.1.8 (BONUS)**: Implement your linear regression model as a class.

> **Ex. 12.1.9 (BONUS)**: Is it possible to adjust our linear model to become a Lasso? Is there a simple fix?

## Exercise Section 12.2: Houseprices
In this example we will try to predict houseprices using a lot of variable (or features as they are called in Machine Learning). We are going to work with Kaggle's dataset on house prices, see information [here](https://www.kaggle.com/c/house-prices-advanced-regression-techniques). Kaggle is an organization that hosts competitions in building predictive models.

> **Ex. 12.2.0:** Load the california housing data with scikit-learn using the code below. Inspect the data set. 

In [1]:
# [Answer to Ex. 12.2.0]
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

cal_house = fetch_california_housing()    
X = pd.DataFrame(data=cal_house['data'], 
                 columns=cal_house['feature_names'])\
             .iloc[:,:-2]
y = cal_house['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.5, random_state=1)
X.isnull().sum()
X.shape
X.describe()
X

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467
5,4.0368,52.0,4.761658,1.103627,413.0,2.139896
6,3.6591,52.0,4.931907,0.951362,1094.0,2.128405
7,3.1200,52.0,4.797527,1.061824,1157.0,1.788253
8,2.0804,42.0,4.294118,1.117647,1206.0,2.026891
9,3.6912,52.0,4.970588,0.990196,1551.0,2.172269




> **Ex.12.2.1**: Generate interactions between all features to third degree, make sure you **exclude** the bias/intercept term. How many variables are there? Will OLS fail? 

> After making interactions rescale the features to have zero mean, unit std. deviation. Should you use the distribution of the training data to rescale the test data?  

>> *Hint 1*: Try importing `PolynomialFeatures` from `sklearn.preprocessing`

>> *Hint 2*: If in doubt about which distribution to scale, you may read [this post](https://stats.stackexchange.com/questions/174823/how-to-apply-standardization-normalization-to-train-and-testset-if-prediction-i).

In [2]:
# [Answer to Ex. 12.2.1]
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import StandardScaler


X_train_p = PolynomialFeatures(degree=3, include_bias=False).fit_transform(X_train)
X_test_p = PolynomialFeatures(degree=3, include_bias=False).fit_transform(X_test)

scaler = StandardScaler()
scaled_x_train = scaler.fit_transform(X_train_p)
scaled_x_train
scaled_x_test = scaler.transform(X_test_p)
scaled_x_test

array([[-0.33298175,  0.83025875, -0.34866789, ..., -0.04726926,
        -0.01443505, -0.01349305],
       [-1.01117619,  0.67200908, -0.16955442, ..., -0.04210401,
        -0.01424053, -0.01348811],
       [ 0.07406292,  1.38413261, -0.35712192, ..., -0.01896188,
        -0.01359914, -0.01347342],
       ...,
       [ 0.44596052,  0.98850843, -0.43205052, ..., -0.04983522,
        -0.01445712, -0.01349298],
       [ 0.30643892, -0.98961249,  0.1913317 , ..., -0.02163071,
        -0.01393352, -0.01348493],
       [ 1.011247  , -0.91048766,  0.10956687, ..., -0.03399015,
        -0.01428989, -0.01349191]])

> **Ex.12.2.2**: Estimate the Lasso model on the train data set, using values of $\lambda$ in the range from $10^{-4}$ to $10^4$. For each $\lambda$  calculate and save the Root Mean Squared Error (RMSE) for the test and train data. 

> *Hint*: use `logspace` in numpy to create the range.


In [13]:
# [Answer to Ex. 12.2.2]
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.metrics import mean_squared_error as mse
import numpy as np
from math import sqrt

perform = []
lambdas = np.logspace(-4, 4, 17)

for lambda_ in lambdas:
    lasso = Lasso(alpha=lambda_, random_state=1)
    lasso.fit(X_train_p, y_train) # fit on the training data
    y_pred = lasso.predict(X_test_p) # validate on the test data
    perform.append(sqrt(mse(y_pred, y_test)))
    
hyperparam_perform = pd.Series(perform,index=lambdas)

optimal = hyperparam_perform.nsmallest(1)    
print('Optimal lambda:', optimal.index[0])
print('Test RMSE: %.3f' % optimal.values[0])

Optimal lambda: 0.31622776601683794
Test RMSE: 0.999


> **Ex.12.2.3**: Make a plot with on the x-axis and the RMSE measures on the y-axis. What happens to RMSE for train and test data as $\lambda$ increases? The x-axis should be log scaled. Which one are we interested in minimizing? 

> Bonus: Can you find the lambda that gives the lowest MSE-test score?

In [2]:
# [Answer to Ex. 12.2.3]